## 1 加载包

In [1]:
library(stm)
library(tidyverse)
library(jiebaR)
library(quanteda)
library(showtext)
library(showtextdb)
# 自动使用 showtext 渲染
showtext_auto()

stm v1.3.7 successfully loaded. See ?stm for help. 
 Papers, resources, and other materials at structuraltopicmodel.com



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: jiebaRD



Package version: 4.0.2
Unicode version: 14.0
ICU version: 70.1



Parallel computing: 4 of 4 threads used.



See https://quanteda.io for tutorials and examples.



Loading required package: sysfonts



Loading required package: showtextdb



## 2 整理数据

In [2]:
#导入数据
data1 <- read.csv(file = "/kaggle/input/dou-data1007/douban_data.csv")


In [3]:
glimpse(data1)

Rows: 108,985
Columns: 10
$ 标题          <chr> "# 阿尔茨海默病：一份温馨的护理指南", ".", ".", "/", "/"…
$ year          <dbl> 2024, 2023, 2023, 2021, 2021, 2021, 2021, 2021, 2021, 20…
$ ip            <chr> "上海", "北京", "海南", "", "", "", "", "", "", "", "", …
$ text          <chr> "# 阿尔茨海默病：一份温馨的护理指南\n      #阿尔茨海默病…
$ diqu          <dbl> 1, 1, 1, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ zuwu          <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ text1         <chr> "阿尔茨海默病一份温馨的护理指南阿尔茨海默病一份温馨的护…
$ text1_cut     <chr> "病 一份 护理 指南 病 一份 护理 指南 豆瓣 充满 文艺 气息…
$ text1_cleaned <chr> "病 一份 护理 指南 病 一份 护理 指南 豆瓣 充满 文艺 气息…
$ text2_cleaned <chr> "病 一份 护理 指南 病 一份 护理 指南 充满 气息 社区 分享…


In [4]:
# 从 data1 中选择指定列，并按标题、year 和 ip 合并相同的行，最后重命名列名
data <- data1 %>%
  select(标题, year, diqu, text2_cleaned) %>%
  group_by(标题, year, diqu) %>%
  summarise(
    text = paste(text2_cleaned, collapse = " "),  # 合并 text2_cleaned 并重命名为 text
    .groups = "drop"  # 解除分组
  )


In [5]:
glimpse(data)

Rows: 9,253
Columns: 4
$ 标题 <chr> "# 阿尔茨海默病：一份温馨的护理指南", ".", "/", "0", "00后提前经…
$ year <dbl> 2024, 2023, 2021, 2021, 2022, 2023, 2024, 2024, 2021, 2020, 2021,…
$ diqu <dbl> 1, 1, NA, NA, NA, 1, 1, 2, NA, NA, NA, 1, 2, 3, NA, 1, 2, 1, 2, 4…
$ text <chr> "病 一份 护理 指南 病 一份 护理 指南 充满 气息 社区 分享 感悟 知…


In [6]:
table(data$year,useNA="ifany")


2019 2020 2021 2022 2023 2024  NaN <NA> 
  41  548 1880 2579 2667 1506   30    2 

In [7]:
# 过滤掉 year 列中为 NaN 或 NA 的行
data <- data %>%
  filter(!is.na(year) & !is.nan(year))

# 检查结果
table(data$year, useNA = "ifany")


2019 2020 2021 2022 2023 2024 
  41  548 1880 2579 2667 1506 

In [8]:
table(data$diqu, useNA = "ifany")


   1    2    3    4 <NA> 
2436 1914  532  458 3881 

## 3 准备数据

In [9]:
# 4. Text processing using textProcessor (since the text is already segmented)
processed <- textProcessor(documents = data$text, 
                           metadata = data, 
                           removestopwords = FALSE,
                           removepunctuation = FALSE,
                           stem = FALSE,
                          wordLengths = c(2,Inf))



Building corpus... 
Converting to Lower Case... 
Removing numbers... 
Creating Output... 


In [10]:
# 5. Prepare documents for STM without removing low frequency terms
out <- prepDocuments(processed$documents, processed$vocab, processed$meta)
print(head(out$documents))  # View the first few processed documents

Removing 170 of 12986 terms (170 of 482905 tokens) due to frequency 
Your corpus now has 9026 documents, 12816 terms and 482735 tokens.

$`1`
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]   45  149  192  245  654  918 1054 1238 1320  1460  1478  1539  1629  1775
[2,]    2    2    2    1    2    1    1    1    1     1     1     1     1     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]  1867  2241  2289  2397  2474  2535  2759  2953  2987  3332  3719  3724
[2,]     2     1     1     5     1     2     1     1     1     1     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]  3731  4594  4604  4641  4642  4704  4750  4855  4879  4948  5112  5114
[2,]     1     1     1     1     1     2     2     1     1     1     1     2
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]  5161  5320  5598  5616  5742  5841  5844  5855  5861  5866  5959  5978
[2,]     1     2     1     1     2     1    15     1     1     3     1     3
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,6

In [11]:
# out$documents格式为：每条文本分词对应词汇表vocab中的id，以及词频
head(out$documents, 5)
head(out$vocab, 200)
head(out$meta, 5)

$`1`
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]   45  149  192  245  654  918 1054 1238 1320  1460  1478  1539  1629  1775
[2,]    2    2    2    1    2    1    1    1    1     1     1     1     1     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]  1867  2241  2289  2397  2474  2535  2759  2953  2987  3332  3719  3724
[2,]     2     1     1     5     1     2     1     1     1     1     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]  3731  4594  4604  4641  4642  4704  4750  4855  4879  4948  5112  5114
[2,]     1     1     1     1     1     2     2     1     1     1     1     2
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]  5161  5320  5598  5616  5742  5841  5844  5855  5861  5866  5959  5978
[2,]     1     2     1     1     2     1    15     1     1     3     1     3
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]  5981  5988  6013  6321  6336  6392  6603  6810  6830  6868  7018  7119
[2,]     1     1     1     1     1    10     3     1     3     1     3     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]  7161  7168  7250  7253  7270  7350  7375  7528  7933  8002  8335  8532
[2,]     1     1     1     1     1     1     1     3     1     1     2     2
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]  8549  8552  8573  8649  8715  8758  8801  9051  9069  9390  9431  9472
[2,]     4     1     1     1     1     3     4     4     1     3     1     1
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]  9483  9514  9713  9802 10013 10026 10058 10294 10388 10660 10661 10847
[2,]     3     1     1     2     1     1     2     3     1     1     1     1
     [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,] 10927  11047  11058  11114  11134  11138  11180  11182  11235  11555
[2,]     1      2      1      1      2      4      1      1      1      1
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]  11617  11794  11962  11984  11989  12311  12463  12495  12511  12806
[2,]      2      1      1      2      1      2      1      1      1      2

$`2`
     [,1] [,2]  [,3]  [,4]  [,5]  [,6]  [,7]
[1,] 5151 9246 10453 11183 11271 12231 12263
[2,]    2    1     2     1     1     1     1

$`3`
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]   42   76  108  109  128  163  169  205  227   238   286   312   370   391
[2,]    3    9    6    3    9    3    9    3    3     9     3     3     3     9
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]   407   446   488   502   512   526   540   561   591   606   620   624
[2,]     6     6     9    12     3     6     3     9     6     6     6    12
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]   635   679   703   708   711   736   738   745   843   849   864   867
[2,]     3     3     3     3    27     3    15     3     3     6     3     3
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]   912   941   950  1050  1068  1079  1110  1135  1219  1232  1247  1328
[2,]     3     3     3     3     9     6     3     3     3     3     6     3
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]  1338  1404  1461  1472  1474  1480  1517  1534  1542  1559  1628  1642
[2,]     3     6     3     3     3     6     3     3     3     3     3     3
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]  1797  1820  1822  1852  1865  1918  1995  2056  2063  2078  2137  2218
[2,]     3     3     3     3     3     3     6     3     3     3     9     3
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]  2247  2302  2305  2333  2359  2385  2409  2413  2415  2439  2542  

[1] "一丁点"   "一万"     "一万个"   "一万二"   "一万五"   "一万元"  
  [7] "一万块"   "一万多"   "一万左右" "一万步"   "一下下"   "一下子"  
 [13] "一丝"     "一两"     "一两万"   "一两个"   "一两千"   "一两句"  
 [19] "一两周"   "一两天"   "一两岁"   "一两年"   "一两次"   "一两点"  
 [25] "一两百"   "一两百万" "一个个"   "一个包"   "一个半"   "一个半月"
 [31] "一个多"   "一个多月" "一个愿打" "一个月"   "一个样"   "一个点"  
 [37] "一个班"   "一个男孩" "一事"     "一二"     "一代"     "一代人"  
 [43] "一代代"   "一件"     "一份"     "一会"     "一会儿"   "一位"    
 [49] "一体"     "一個"     "一倍"     "一儿"     "一出"     "一刀"    
 [55] "一分"     "一分一毫" "一分钟"   "一分钱"   "一刻"     "一副"    
 [61] "一包"     "一千"     "一千万"   "一千五"   "一千块"   "一千多"  
 [67] "一千左右" "一半"     "一半一半" "一半儿"   "一半多"   "一厅"    
 [73] "一双"     "一发"     "一口"     "一句"     "一只"     "一台"    
 [79] "一名"     "一员"     "一周"     "一味"     "一回"     "一团"    
 [85] "一圈"     "一场"     "一块"     "一块钱"   "一堆"     "一声"    
 [91] "一处"     "一多"     "一夜"     "一大"     "一大半"   "一大块"  
 [97] "一大堆"   "一大截"   "一大笔"   "一大部分" "一天两天" "一天天"  
[103] "一头"     "一套"     "一套套"   "一女"     "一室"     "一家"    
[109] "一家人"   "一家子"   "一对"     "一对一"   "一封"     "一小块"  
[115] "一小部分" "一层"     "一层楼"   "一岁"     "一带"     "一帮"    
[121] "一幕"     "一平"     "一年"     "一年半载" "一床"     "一座"    
[127] "一弟"     "一张"     "一户"     "一所"     "一手"     "一打"    
[133] "一批"     "一把"     "一招"     "一拳"     "一挺"     "一排"    
[139] "一搏"     "一摊"     "一支"     "一整天"   "一斤"     "一方"    
[145] "一日"     "一日三餐" "一晃"     "一月"     "一本"     "一朵"    
[151] "一条"     "一杯"     "一枚"     "一架"     "一栋"     "一根"    
[157] "一桌"     "一档"     "一棵"     "一楼"     "一次次"   "一款"    
[163] "一步"     "一步步"   "一段"     "一毛钱"   "一流"     "一滴"    
[169] "一点"     "一点一点" "一点儿"   "一点多"   "一点点"   "一瓶"    
[175] "一生"     "一百"     "一百万"   "一百个"   "一百块"   "一百多"  
[181] "一百多万" "一百多元" "一百多块" "一百岁"   "一盆"     "一盒"    
[187] "一盘"     "一眼"     "一瞬间"   "一碗"     "一碗水"   "一种"    
[193] "一秒"     "一程"     "一站"     "一笔"     "一等"     "一箱"    
[199] "一篇"     "一米"

标题                               year diqu
1 # 阿尔茨海默病：一份温馨的护理指南 2024  1  
2 .                                  2023  1  
3 /                                  2021 NA  
4 0                                  2021 NA  
5 00后提前经历养老                   2022 NA  
  text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

## 4 searchk

In [12]:
#另一个notebook

## 5 拟合13

In [13]:
stm_13 <- stm(documents = out$documents, vocab = out$vocab, K = 13, prevalence = ~year, data = out$meta, max.em.its = 75, verbose = FALSE)
summary(stm_13)

A topic model with 13 topics, 9026 documents and a 12816 word dictionary.


Topic 1 Top Words:
 	 Highest Prob: 医院, 老年人, 医生, 治疗, 手术, 情况, 护工 
 	 FREX: 透析, 血管, 陪诊, 种植, 造影, 照护, 针灸 
 	 Lift: 一针, 三针, 专家门诊, 临床经验, 五十多年, 交叉感染, 低质量 
 	 Score: 医院, 医生, 脑梗, 治疗, 护工, 病人, 老年人 
Topic 2 Top Words:
 	 Highest Prob: 退休, 喜欢, 回家, 地方, 做饭, 上班, 衣服 
 	 FREX: 打扫, 洗碗, 做饭, 房间, 快递, 买菜, 扫地 
 	 Lift: 个游, 保洁员, 保育, 信群, 卖菜, 小区业主, 开厂 
 	 Score: 退休, 做饭, 家务, 衣服, 喜欢, 回家, 上班 
Topic 3 Top Words:
 	 Highest Prob: 结婚, 离婚, 男人, 婆婆, 婚姻, 儿子, 男方 
 	 FREX: 扶贫, 离婚, 绝食, 结了婚, 男方, 出轨, 恋爱 
 	 Lift: 绝食, 下毒, 个富, 了不得, 产生分歧, 人情味, 俩家 
 	 Score: 离婚, 结婚, 男方, 男人, 婚姻, 儿子, 彩礼 
Topic 4 Top Words:
 	 Highest Prob: 保险, 医疗, 医疗险, 报销, 体检, 一年, 百万 
 	 FREX: 医疗险, 百万, 意外险, 商业, 疾险, 保费, 理赔 
 	 Lift: 一百多元, 三五千, 不赔, 丙类, 主险, 享生, 保司 
 	 Score: 医疗险, 保险, 医疗, 意外险, 疾险, 体检, 百万 
Topic 5 Top Words:
 	 Highest Prob: 手机, 时间, 视频, 朋友, 老爸, 求助, 发现 
 	 FREX: 访谈, 点滴, 戒烟, 下载, 尿失禁, 欢乐, 电子产品 
 	 Lift: 饮用, 招募, 消消, 电子产品, 访谈, 不学, 两包 
 	 Score: 戒烟, 手机, 视频, 尿失禁, 访谈, 老爸, 老妈 
Topic 6 Top Words:
 	 Highest Prob: 父母, 买房, 房子, 毕业, 老家, 不用, 结婚 
 	 FREX: 出国, 房价, 定居, 买房

In [14]:
effects_13 <- estimateEffect(1:13 ~ year, stm_13, meta = out$meta)
summary(effects_13)


Call:
estimateEffect(formula = 1:13 ~ year, stmobj = stm_13, metadata = out$meta)


Topic 1:

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -55.297200   3.859983  -14.33   <2e-16 ***
year          0.027401   0.001909   14.36   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Topic 2:

Coefficients:
              Estimate Std. Error t value Pr(>|t|)
(Intercept)  1.9656930  2.1445515   0.917    0.359
year        -0.0009378  0.0010604  -0.884    0.377


Topic 3:

Coefficients:
             Estimate Std. Error t value Pr(>|t|)   
(Intercept)  8.074366   2.461776   3.280  0.00104 **
year        -0.003966   0.001217  -3.258  0.00113 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Topic 4:

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 13.960247   4.099046   3.406 0.000663 ***
year        -0.006862   0.002027  -3.385 0.000714 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0

In [15]:
# 将所有当前环境中的对象保存到指定位置
save.image('/kaggle/working/stm_douban_13.RData')
